## 3. Flask Annotator App for Concordance Line Annotation & Analysis

- This notebook demonstrates how to use an AI prompt engineering Flask App for quick annotation of concordance lines obtained from KWIC searches on various corpora (e.g., COCA, GloWbE, EcoLexicon).

In [3]:
ls -la ../../annotator_app/

total 208
drwxr-xr-x  11 ejacquot  staff    352 Nov 25 19:54 ./
drwxr-xr-x  29 ejacquot  staff    928 Nov 25 19:54 ../
-rw-r--r--@  1 ejacquot  staff   6148 Nov 15 23:48 .DS_Store
drwxr-xr-x   6 ejacquot  staff    192 Nov 25 19:54 .venv/
-rw-r--r--   1 ejacquot  staff   2363 Nov 25 19:57 README.md
-rw-r--r--   1 ejacquot  staff   4063 Nov 17 01:35 README_UPLOAD.md
drwxr-xr-x  19 ejacquot  staff    608 Nov 24 11:55 annotations/
-rw-r--r--   1 ejacquot  staff  83517 Nov 24 14:13 app.py
drwxr-xr-x   2 ejacquot  staff     64 Nov 14 15:37 llm_annotations/
-rw-r--r--   1 ejacquot  staff    659 Nov 25 19:54 settings.json
drwxr-xr-x  17 ejacquot  staff    544 Nov 24 11:39 templates/


In [4]:
import sys
import subprocess
import threading
from pathlib import Path
import time

# Run the Flask app at ./annotator_app/app.py without blocking the notebook kernel.
# This cell starts the script as a subprocess and streams its stdout/stderr.

app_path = Path("../..//annotator_app/app.py").resolve()
if not app_path.exists():
    raise FileNotFoundError(f"{app_path} not found. Check the path and try again.")

# Start the app using the same Python interpreter as the notebook
proc = subprocess.Popen(
    [sys.executable, str(app_path)],
    cwd=str(app_path.parent),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

print(f"Started process PID={proc.pid} for {app_path}")

# Background thread to stream process output to the notebook
def _stream_output(p):
    try:
        for line in p.stdout:
            print(line, end="")
    except Exception as e:
        print("Error streaming output:", e)

thread = threading.Thread(target=_stream_output, args=(proc,), daemon=True)
thread.start()

# Convenience: helper functions to check status and stop the process
def is_running():
    return proc.poll() is None

def stop_app(timeout=5):
    if not is_running():
        print("Process is not running.")
        return
    print("Terminating process...")
    proc.terminate()
    try:
        proc.wait(timeout=timeout)
    except subprocess.TimeoutExpired:
        print("Process did not exit, killing...")
        proc.kill()
    print("Process stopped.")

# Example usage:
# is_running()          -> returns True if app is still running
# stop_app()            -> gracefully stop the app (force-kills if needed)
# proc.pid              -> access the process id
#
# Note: If the Flask app binds to a port (e.g., 5000), open http://localhost:5000 in your browser.
time.sleep(0.1)  # small sleep so the cell returns after process started

Started process PID=2809 for /Users/ejacquot/Documents/Github/getout_of_text_3/annotator_app/app.py


📁 Exports directory: /Users/ejacquot/Documents/Github/getout_of_text_3/exports
📝 Annotations directory: /Users/ejacquot/Documents/Github/getout_of_text_3/annotator_app/annotations
📊 Available files: []
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat
📁 Exports directory: /Users/ejacquot/Documents/Github/getout_of_text_3/exports
📝 Annotations directory: /Users/ejacquot/Documents/Github/getout_of_text_3/annotator_app/annotations
📊 Available files: []
 * Debugger is active!
 * Debugger PIN: 682-672-559
127.0.0.1 - - [25/Nov/2025 20:01:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2025 20:01:20] "GET /favicon.ico HTTP/1.1" 404 -


_______________________
## Troubleshoot

- Restart the kernel if the Flask app does not start properly.
- Check for running PIDs on the same port:

In [5]:
# check what is running on port 5001
!lsof -i :5001

In [4]:
!kill -9 $(lsof -t -i:5000) || echo "No process on port 5000"

In [5]:
!kill -0 {proc.pid} && echo "Process {proc.pid} is running." || echo "Process {proc.pid} is not running."

Process 52942 is running.
